In [2]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("Module 5 Zoomcamp") \
    .getOrCreate()

24/03/01 04:17:51 WARN Utils: Your hostname, codespaces-be4276 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/01 04:17:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 04:17:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/03/01 04:18:11 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


#### Question 1. Spark version

In [8]:
spark.version

'3.5.1'

#### Question 2. FHV October 2019 partition size

In [6]:
# CSV to Spark DataFrame
csv_file = "data/fhv_tripdata_2019-10.csv"
df = spark.read.csv(csv_file, header=True, inferSchema=True)

In [7]:
# Repartition DataFrame to 6 Partitions
df_rpt = df.repartition(6)

# DataFrame to Parquet
parquet_output = "data/output"
df_rpt.write \
    .mode("overwrite") \
    .parquet(parquet_output)


#### Question 3. Count records on 15th of October

In [9]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PUlocationID', IntegerType(), True), StructField('DOlocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [10]:
from pyspark.sql.functions import col

In [15]:
count_rows = df.filter(col("pickup_datetime").cast("date") == '2019-10-15').count()

In [16]:
count_rows

62610

#### Question 4. The longest trip

In [18]:
from pyspark.sql.functions import col, unix_timestamp, hour, expr

In [19]:
df = df.withColumn("pickup_timestamp", unix_timestamp(col("pickup_datetime")).cast("timestamp")) \
    .withColumn("dropoff_timestamp", unix_timestamp(col("dropoff_datetime")).cast("timestamp"))
    
df = df.withColumn("duration", (col("dropoff_timestamp").cast("long") - col("pickup_timestamp").cast("long")) / 3600)
max_duration = df.agg({"duration": "max"}).collect()[0][0]
max_duration

631152.5

#### Question 5. Spark UI port

4040

#### Question 6. Least frequent pickup location zone 

In [22]:
# Read CSV data into DataFrame
csv_file_path = "data/taxi_zone_lookup.csv"
df_zone = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [23]:
df.head()

Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 23), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 35), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00009', pickup_timestamp=datetime.datetime(2019, 10, 1, 0, 23), dropoff_timestamp=datetime.datetime(2019, 10, 1, 0, 35), duration=0.2)

In [25]:
df_zone.head()

Row(LocationID=1, Borough='EWR', Zone='Newark Airport', service_zone='EWR')

In [26]:
joined_df = df.join(df_zone, df.PUlocationID == df_zone.LocationID, "left")

result_df = joined_df.groupBy("Zone").count()
result_df = result_df.orderBy("count")

result_df.show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
|       Broad Channel|    8|
|     Highbridge Park|   14|
|        Battery Park|   15|
|Saint Michaels Ce...|   23|
|Breezy Point/Fort...|   25|
|Marine Park/Floyd...|   26|
|        Astoria Park|   29|
|    Inwood Hill Park|   39|
|       Willets Point|   47|
|Forest Park/Highl...|   53|
|  Brooklyn Navy Yard|   57|
|        Crotona Park|   62|
|        Country Club|   77|
|     Freshkills Park|   89|
|       Prospect Park|   98|
|     Columbia Street|  105|
|  South Williamsburg|  110|
+--------------------+-----+
only showing top 20 rows

